# TwitchYoutubeChatAnalysis

###### Incardona Biagio X81000608

## Cos'è?

Twitch-Youtube-Chat-Analysis è un progetto universitario realizzato per il corso **TECHNOLOGIES FOR ADVANCED PROGRAMMING** dallo studente Incardona Biagio, matricola **X81000608**.
Il progetto consiste nell'analisi in real time delle interazioni degli spettatori di un live stream su YouTube e/o Twitch.

Il progetto è stato realizzato in modo da essere facilmente estendibile a nuove piattaforme.

### #MaiUnaGioia

<p align="center">
  <img src="./progetto_meme.jpg" width="400" title="hover text">
</p>

## Per chi?

Il progetto è mirato a tutti gli streamer, specialmente per coloro che abitualmente effettuano stream su più piattaforme in parallelo, in questo caso YouTube e Twitch.

Tale progetto può anche essere utilizzato da potenziali investitori (brand in cerca di pubblicità) al fine da evitare di investire su profili con views comprate o con un basso impatto sul pubblico.

<p align="center">
  <img src="./Perche_usarlo.jpg" width="400" title="hover text">
</p>


## Obbiettivo

Lo scopo ultimo di tale progetto è quello di avere un'interfaccia semplificata per analizzare il comportamento degli spettatori, in quanto le chat in una live stream sono spesso molto confusionare.
Tale problematica viene ulteriormente amplificata se si effettuano gli stream in parallelo su più piattaforme.

Verrà quindi fornita una dashboard contenente le analisi principali con la possibilità:
  1. Avere un'analisi riassuntiva riguardante entrambe le piattaforme contemporaneamente
  2. Avere un'analisi mirata alla singola piattaforma

## Struttura

il progetto può essere suddiviso in 5 fasi separate:
    
   1. [Ingestion](#Ingestion)
   2. [Streaming](#Streaming)
   3. [Processing](#Processing)
   4. [Indexing](#Indexing)
   5. [Visualization](#Visualization)
    
<p align="center">
  <img src="./Schema.png" width="600" title="hover text">
</p>

## Ingestion

La fase di ingestion dei dati consiste nella raccolta degli stessi da fonti esterne, in questo caso le nostre fonti saranno YouTube e Twitch.

Al fine di ottenere una semplice estendibilità del progetto si è scelto di raccogliere questi dati tramite le API o tramite web scraping ed in seguito incanalare tutti i dati verso un'unica piattaforma : **Logstash**.


### Twitch

Per raccogliere i dati di Twitch si sono utilizzate le API fornite dallo stesso.

Si effettuano richieste in loop ed i dati ricevuti vengono inseriti in un json ed in fine inviati a Logstash.

Al fine di ottenere una consistenza temporale senza l'interferenza della latenza delle varie tecnologie utilizzate si è deciso di inserire in questa fase il timestamp di quando un dato è arrivato

### YouTube

Nonostante YouTube fornisca delle ottime API, questo è troppo rispettoso della privacy per i nostri gusti, non permette infatti di accedere alle risorse di una live chat ammenoché non sia la nostra e comunque sotto esplicito consenso dell'utente fornito tramite interfaccia grafica. Ma questo blocco ha fatto scattare la nostra carta trappola: **Selenium**

<p align="center">
  <img src="./selenium_meme.jpg" width="600" title="hover text">
</p>

#### Selenium

Selenium è il più diffuso tool per l’automazione di browser in commercio. Nonostante la sua principale funzione non sia quella di web testing automatico, esso viene utilizzato principalmente per questo.

Tale prodotto con le opportune impostazioni può diventare un ottimo Web Scraper per siti web dinamici, impossibili da analizzare con un classico web scraper.

Selenium ci permette non solo di analizzare il sorgente di una pagina web, ma anche il codice creato dopo l'esecuzione degli script, in questo caso in JavaScript.

Per far ciò si è scelto di usare i driver di Google Chrome in modo da emulare esattamente il comportamento del browser web.

<p align="center">
  <img src="./meme_google.jpg" width="600" title="hover text">
</p>

#### Vediamo come

```python
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-extensions')
    browser = webdriver.Chrome(chrome_options=chrome_options)
    browser.get("https://www.youtube.com/live_chat?v=" + channel)
    chats = []
    count = 0
    rate = 0.0
    start_time = time.time()
    while True:
        for chat in browser.find_elements_by_css_selector('yt-live-chat-text-message-renderer'):
            usr = chat.find_element_by_css_selector("#author-name").get_attribute('innerHTML')
            mex = chat.find_element_by_css_selector("#message").get_attribute('innerHTML')
            usr = str(usr).split('<')[0]

            obj = json.dumps({'author_name': usr, 'message': mex})
            x = json.loads(obj)
```

### Un po di preprocessing

Uno dei nostri obbiettivi è quello di estrapolare l'engagement sul pubblico.

Per il nostro scopo definiamo engagement come il tempo trascorso tra l'invio di un messaggio e un altro.

Tale valore verrà calcolato con la seguente funzione

```python
rate = 6/(end_time-start_time)
```

### Logstash

Logstash è uno strumento per raccogliere, elaborare e inoltrare eventi e messaggi di registro. La raccolta viene eseguita tramite plug-in di input configurabili. Una volta che un plug-in di input ha raccolto i dati, può essere elaborato da qualsiasi numero di filtri che modificano e annotano i dati dell'evento. Infine Logstash indirizza gli eventi ai plugin di output che possono inoltrare gli eventi a una varietà di programmi esterni tra cui Elasticsearch, file locali, Kafka e altri.

Per il nostro scopo Logstash raccoglierà i dati dai due scraper in formato json tramite una connessione TCP e li inverà in seguito a Kafka usando il topic "data".

Per permettere il passaggio dei dati tra Logstash e i due scraper serve un piccolo protocollo in modo da inviare dati omogenei.

```
{
    'username' : username dell utente,
    'message' : contenuto del messaggio,
    'engagement' : engagement ,
    'timestamp' : timestamp di quando un messaggio e arrivato,
    'source' : nome della sorgente
}
```

## Streaming

Per effettuare il data streaming ci si è appoggiati ad Apache Kafka.

### Apache Kafka

Apache Kafka è una piattaforma per il data streaming distribuita che permette di pubblicare, sottoscrivere, archiviare ed elaborare flussi di record in tempo reale. È progettata per gestire flussi di dati provenienti da più fonti distribuendoli a più consumatori. In breve, consente di spostare grandi quantità di dati da un punto qualsiasi a un altro nello stesso momento.

## Processing

Una volta raccolti i dati tramite kafka vogliamo manipolare e ottenere informazioni aggiuntive da questi dati.

A tal fine ci si è appoggiati a Apache Spark.

### Apache Spark

Apache Spark è un framework per il calcolo distribuito che risulta 100 volte più veloce rispetto al suo principale concorrente (MapReduce).

Spark è ottimizzato per algoritmi di machine learning, risulta ovvio quindi che la nostra scelta ricade su di lui.

Per la creazione e la manipolazione di DataFrame on the fly si è fatto uso di PySpark, delle API fornite da Spark per interfacciarsi a lui tramite Python.

<p align="center">
  <img src="./pyspak_errore.jpg" width="600" title="hover text">
</p>

### Machine Learning

Appoggiandoci a Spark possiamo applicare algoritmi di ML in maniera molto rapida.

I nostri obbiettivi sono 2:
    
    1. Sentiment analysis delle chat
    2. Keyword extraction delle chat
    
#### 1. Sentiment analysis

<p align="center">
  <img src="./scelta_vader.jpg" width="600" title="hover text">
</p>


In [12]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
sid = SentimentIntensityAnalyzer()

def get_sentiment(text):
    value = sid.polarity_scores(text)
    value = value['compound']
    return value

print(get_sentiment("i'm ugly :("))
print(get_sentiment("i'm pretty :)"))

-0.7351
0.7351


#### 2. Keyword extraction

Per effettuare l'estrazione delle keyword in ogni messaggio si è fatto uso di una delle più potenti librerie di multi language natural language processing attualmente esistenti per Python : **SpaCy**.

Fornendo inoltre 3 versioni della libreria : small(11MB), medium(~100MB) e large(~1GB) permette di essere adattata anche ad eventuali problematiche legate alla memoria.

Noi ovviamente abbiamo utilizzato la versione large.

##### Code
```python
def get_keyword(text):
    result = []
    pos_tag = ['PROPN', 'ADJ', 'NOUN']
    doc = nlp(text.lower())
    for token in doc:
        if(token.text in nlp.Defaults.stop_words or token.text in punctuation):
            continue
        if(token.pos_ in pos_tag):
            result.append(token.text)

    back = [x for x in Counter(result).most_common(1)]       
    if len(back) == 0:
        return None 
    return back[0][0]
```

---
---

Alla fine di tutto il verrà fornito un file JSON strutturato nel seguente modo

```
{
    'username' : username dell utente,
    'timestamp' : timestamp di quando un messaggio e arrivato,
    'mex' : contenuto del messaggio,
    'engagement' : engagement,
    'source' : nome della sorgente,
    'mex_sentiment' : valore della sentiment analysis del messaggio,
    'keyword' : keyword rappresentativa del messaggio
}
```

## Indexing

Indicizzeremo i nostri dati in modo da abbattere i costi per la ricerca. A tal fine faremo uso di **ElasticSearch**.

### ElasticSearch

ElasticSearch è un motore di ricerca basato su Lucene, le API di reperimento dati Open Source più veloce e utilizzato per la realizzazione dei motori di ricerca.

Tutte le funzionalità sono nativamente esposte tramite interfaccia RESTful, mentre le informazioni sono gestite come documenti JSON.

Nel progetto sfrutteremo un solo indice : "data"

L'utilizzo di un solo indice ci permette di passare da una piattaforma all'altra o lavorare insieme in maniera molto semplice ed intuitiva

## Visualization

Kibana è una dashboard di visualizzazione dati open source per Elasticsearch.
Fornisce funzionalità di visualizzazione dei contenuti indicizzati su Elasticsearch. Gli utenti possono creare grafici a barre, a linee e a torta e mappe su grandi volumi di dati.

Oltre ad effettuare grafici, kibana permette l'elaborazione di dati in tempo reale!

<p align="center">
  <img src="./dashboard.png" width="1000" title="hover text">
</p>

---
---
---

# Idee per il futuro

* inserire nuove piattaforme di streaming

# GRAZIE PER L'ATTENZIONE

<p align="center">
  <img src="./saluti_meme.jpg" width="600" title="hover text">
</p>